### Part 1. Scrapping restaurant data from Yelp Fusion API

In [1]:
import yelp
import requests
import json
from datetime import datetime

In [4]:
client_id='vbSXv5AR9CFLmp9Q6iDgkw'
api_key='EeDkyCyJJpe_YfI7U9EGg2uL7QrmMjgA5fPwbDD7xecjPBEeCUlFRr_yla2j20copss7AxRh7JcYJJdPHvKK_emSHbiMh1nAOWCuIxka5vWIVRzpeqAeKfQWnAdkYXYx'
headers = {'Authorization': 'Bearer %s' % api_key}

In [5]:
# five location, 7 cuisine type
rest_dict = {}
location = ['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten_Island']
cuisine = ['Chinese', 'Japanese', 'Korean', 'Thai', 'Italian', 'Mexican', 'American']

# scrap Yelp by location and cuisine type, get 50 results from each
for l in location:
    for c in cuisine:
        url='https://api.yelp.com/v3/businesses/search'
        params = {'term':c,'location':l,'limit':50}
        req=requests.get(url, params=params, headers=headers)
        data = json.loads(req.text)
        if req.status_code == 200:
            rest_dict[c+l] = data['businesses']
        else:
            print(c + " " + l)

In [9]:
# format scrapped yelp code
# remove potential identical query results by checking id

NY_restaurant = []
id_list = []
for dk in rest_dict:
    cru = ""
    loc = ""
    for c in cuisine:
        if c in dk:
            cru = c
    for l in location:
        if l in dk:
            loc = l
    for rest in rest_dict[dk]:
        if rest['id'] in id_list: continue
        id_list.append(rest['id'])
        samp = {}
        samp['id'] = rest['id']
        samp['name'] = rest['name']
        samp['alias'] = rest['alias']
        samp['cuisine'] = cru
        if rest['location']['city'] in location:
            samp['area'] = rest['location']['city']
        else:
            samp['area'] = loc
        samp['image_url'] = rest['image_url']
        samp['is_closed'] = rest['is_closed']
        samp['url'] = rest['url']
        samp['review_count'] = rest['review_count']
        samp['categories'] = rest['categories']
        samp['rating'] = str(rest['rating'])
        samp['latitude'] = str(rest['coordinates']['latitude'])
        samp['longitude'] = str(rest['coordinates']['longitude'])
        samp['transactions'] = rest['transactions']
        samp['location'] = rest['location']
        samp['display_address'] = ' '.join(rest['location']['display_address'])
        samp['zipcode'] = rest['location']['zip_code']
        samp['phone'] = rest['phone']
        samp['display_phone'] = rest['display_phone']
        samp['insertedAtTimestamp'] = str(datetime.now())
        NY_restaurant.append(samp) 

yelp scrapping result analysis

In [10]:
len(NY_restaurant)

1447

In [14]:
location_dict = {}
for l in location:
    count = 0
    for rest in NY_restaurant:
        if l == rest['area']:
            count += 1
    location_dict[l] = count
location_dict

{'Manhattan': 329,
 'Brooklyn': 321,
 'Queens': 276,
 'Bronx': 268,
 'Staten_Island': 253}

In [33]:
cuisine_dict = {}
for c in cuisine:
    count = 0
    for rest in NY_restaurant:
        if c == rest['cuisine']:
            count += 1
    cuisine_dict[c] = count
cuisine_dict

{'Chinese': 235,
 'Japanese': 220,
 'Korean': 148,
 'Thai': 168,
 'Italian': 225,
 'Mexican': 235,
 'American': 216}

In [16]:
# save yelp scrapping result as json file
yelp = {"yelp-restaurants": NY_restaurant}
with open("yelp.json", "w") as outfile:
    json.dump(yelp, outfile)

### Part 2. Load Data into DynamoDB Table

In [3]:
import boto3
from boto3.dynamodb.conditions import Key
import time

In [18]:
# get DynamoDB resource
# I work this code in local, so I need access key and secret key
# sorry, I can't put this information in the submitted code
dynamodb = boto3.resource('dynamodb', aws_access_key_id='******', 
                          aws_secret_access_key='********',
                          region_name="us-west-2")
list(dynamodb.tables.all())

[dynamodb.Table(name='yelp-restaurants')]

In [19]:
# put data to restaruant table in DynamoDB
yelp_table = dynamodb.Table('yelp-restaurants')
for i in range(len(NY_restaurant)):
    yelp_table.put_item(
        Item=NY_restaurant[i]
    )
    if i % 50 == 0:
        time.sleep(1)

In [163]:
# try a potential query code
response = yelp_table.query(
    KeyConditionExpression=Key('id').eq('ZhjRaJkfCIVopvamUzrEsA')
)
print(response['Items'][0])

{'location': {'country': 'US', 'address3': '', 'address2': '', 'city': 'West Bronx', 'address1': '2329 Arthur Ave', 'display_address': ['2329 Arthur Ave', 'West Bronx, NY 10458'], 'state': 'NY', 'zip_code': '10458'}, 'rating': '4.0', 'insertedAtTimestamp': '2021-10-11 08:52:50.010532', 'is_closed': False, 'transactions': ['pickup', 'delivery'], 'url': 'https://www.yelp.com/biz/san-gennaro-trattoria-restaurant-bar-west-bronx?adjust_creative=vbSXv5AR9CFLmp9Q6iDgkw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=vbSXv5AR9CFLmp9Q6iDgkw', 'cuisine': 'Italian', 'review_count': Decimal('189'), 'name': 'San Gennaro Trattoria Restaurant Bar', 'display_phone': '(718) 562-0129', 'alias': 'san-gennaro-trattoria-restaurant-bar-west-bronx', 'zipcode': '10458', 'display_address': '2329 Arthur Ave West Bronx, NY 10458', 'area': 'Bronx', 'categories': [{'title': 'Italian', 'alias': 'italian'}, {'title': 'Mediterranean', 'alias': 'mediterranean'}], 'longitude': '-73.8889484480023',

### Part 3. Format Data and Upload to OpenSearch

In [2]:
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests.auth import HTTPBasicAuth

In [24]:
# OpenSearch Data only need id, cuisine, and area location
restsearch = []
id_list = []
for rest in NY_restaurant:
    if rest['id'] not in id_list:
        temp = {}
        temp['id'] = rest['id']
        temp['cuisine'] = rest['cuisine']
        temp['area'] = rest['area']
        restsearch.append(temp)
        id_list.append(rest['id'])

# create json file needed for OpenSearch data upload
json_text = ""
for i in range(len(restsearch)):
    json_text += "{ \"create\" : { \"_index\" : \"restaurants\", \"_type\" : \"_doc\", \"_id\" : \"" + str(i+1) + "\"  } }\n"
    json_text += "{ \"id\" : \"" + restsearch[i]['id'] + "\", \"cuisine\" : \"" + restsearch[i]['cuisine'] + "\", \"area\" : \"" + restsearch[i]['area'] + "\" }\n"

f = open("search.json", "w")
f.write(json_text)
f.close()

run the following command in terminal to upload data to OpenSearch for indexing:

curl -XPOST -u "master-user:master-user-password" "search-restaurants-zvk7jnqfg6krgykwbjskmtveiu.us-west-2.es.amazonaws.com/_bulk" --data-binary @search.json -H "Content-Type: application/json"

In [159]:
# try a potentail OpenSearch query

host = 'https://search-restaurants-zvk7jnqfg6krgykwbjskmtveiu.us-west-2.es.amazonaws.com'
index = 'restaurants'
url = str(host + '/' + index + '/' + '_search')
headers = {'Content-Type': "application/json", 'Accept': "application/json"}
data = {
  "size": 20,
  "query": {
    "bool": { "should": [ 
       { "match": { "area": "Bronx" } }, 
       { "match": { "cuisine": "Italian" } } 
    ]}
  }
}

# I hide master-user and user-password
r = requests.get(url, json=data, headers=headers, auth = HTTPBasicAuth('master-user', 'master-user-password'))

response_dict = json.loads(r.text)
response_dict['hits']['hits'][0]['_source']['id']

'phB6HvQyOUEZmvgg9KzRmA'